In [1]:
import torch
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1,2,3"
print(torch.cuda.device_count())

3


In [2]:
from datasets import load_dataset
import itertools

dataset = load_dataset("bavard/personachat_truecased")
        
# train data

train_data = dataset['train'] 
train_history = train_data['history']
train_conv_id = train_data['conv_id']
train_dataset = [train_history[i] for i, conv_id in enumerate(train_conv_id) \
                 if i + 1 != len(train_conv_id) and conv_id != train_conv_id[i + 1]] 

# test data
test_data = dataset['validation'] 
test_history = test_data['history']
test_conv_id = test_data['conv_id']
test_dataset = [test_history[i] for i, conv_id in enumerate(test_conv_id) \
                 if i + 1 != len(test_conv_id) and conv_id != test_conv_id[i + 1]] 


# validation data
validation_dataset = train_dataset[-len(test_dataset) : ]
train_dataset = train_dataset[ : -len(test_dataset)]

train_df = list(itertools.chain.from_iterable(train_dataset))
test_df = list(itertools.chain.from_iterable(test_dataset))
validation_df = list(itertools.chain.from_iterable(validation_dataset))

df = train_df + validation_df + test_df

No config specified, defaulting to: personachat_truecased/full
Found cached dataset personachat_truecased (/home/***/.cache/huggingface/datasets/bavard___personachat_truecased/full/1.0.0/73ee8f1a0d9e42255af5a8301877a2f3ac638e55b1cd9cbccca5ab7e23d2b638)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
def sbert_encoder(df):
    model = SentenceTransformer('sentence-transformers/all-distilroberta-v1')
    model = model.to('cuda')
    
    def get_embedding(sentence):
        return model.encode(sentence)

    embeddings = list(map(get_embedding, df))
    embeddings = np.array(embeddings)
    np.savetxt(f"distil_roberta_embeddings.txt", embeddings)
    return embeddings

In [4]:
embs = sbert_encoder(df)

In [5]:
embs = np.loadtxt(f"distil_roberta_embeddings.txt")

In [6]:
embs.shape

(259572, 768)

In [7]:
embs

array([[-0.0215999 ,  0.01660189,  0.01502401, ..., -0.00828646,
        -0.00148234,  0.0891    ],
       [ 0.01174353, -0.01628386,  0.02573318, ...,  0.05404224,
         0.02910072, -0.00541264],
       [ 0.0440173 ,  0.02031309,  0.00112783, ...,  0.10994507,
         0.02916543,  0.00200748],
       ...,
       [-0.02050706, -0.04231565, -0.02797482, ..., -0.0134701 ,
         0.06195116, -0.05223875],
       [ 0.00287352,  0.06097682, -0.03203836, ...,  0.03529529,
         0.04074401, -0.03268725],
       [ 0.00415624,  0.00401338, -0.06021528, ..., -0.00561223,
         0.07072005, -0.02475802]])